In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] =  "0"

In [ ]:
from vinorm import TTSnorm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline

In [ ]:
from f5tts_wrapper import F5TTSWrapper

# Initialize the F5-TTS wrapper
tts = F5TTSWrapper(
    vocoder_name="vocos",
    # point to  EraX model checkpoint downloaded from HuggingFace
    ckpt_path="path to model_last.pt",
    vocab_file="vocab.txt",
    use_ema=False,
)

In [ ]:
ref_audio_path = "update_213000_ref.wav"

In [ ]:
import IPython
IPython.display.Audio(ref_audio_path)

In [ ]:
ref_text =  "Thậm chí không ăn thì cũng có cảm giác rất là cứng bụng, chủ yếu là cái phần rốn...trở lên. Em có cảm giác khó thở, và ngủ cũng không ngon, thường bị ợ hơi rất là nhiều"
ref_text = TTSnorm(ref_text)
ref_text

In [ ]:
text =  "Trong khi đó, tại một chung cư trên địa bàn P.Vĩnh Tuy (Q.Hoàng Mai), nhiều người sống trên tầng cao giật mình khi thấy rung lắc mạnh nên đã chạy xuống sảnh tầng 1. Cư dân tại đây cho biết, họ chưa bao giờ cảm thấy ảnh hưởng của động đất mạnh như hôm nay."
text_norm =  TTSnorm(text)
text_norm

In [ ]:
# Path to reference audio and output directory
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Preprocess the reference audio just once
# You can provide a reference text, or leave it blank for auto-transcription
tts.preprocess_reference(
    ref_audio_path=ref_audio_path,
    ref_text=ref_text,  # Leave empty for auto-transcription
    clip_short=True  # Clip long audio to ~12 seconds max
)

print(f"Reference audio duration: {tts.get_current_audio_length():.2f} seconds")

In [ ]:
sentences =  [
    gen_text
]
for i, sentence in enumerate(sentences):
    output_path = os.path.join(output_dir, f"generated_non_ema_{i+1}.wav")

    sentence_normed  = TTSnorm(sentence)
    
    # Generate the audio
    tts.generate(
        text=sentence_normed,
        output_path=output_path,
        nfe_step=20,               # Number of denoising steps (32)
        cfg_strength=2.0,          # Classifier-free guidance strength (2)
        speed=1.0,                 # Speed of generated speech (1.0)
        cross_fade_duration=0.15,  # Cross-fade between chunks (0.15)
    )
    
    print(f"Generated: {output_path}")

In [ ]:
import IPython
IPython.display.Audio(f"{output_path}")

In [ ]:
# For applications needing the raw audio data rather than saved files:
audio_array, sample_rate = tts.generate(
    text=gen_text,
    return_numpy=True,
    nfe_step=20,               # Number of denoising steps (32)
    cfg_strength=2.0,          # Classifier-free guidance strength (2)
    speed=1.0,                 # Speed of generated speech (1.0)
    cross_fade_duration=0.15,  # Cross-fade between chunks (0.15)
)
print(f"Generated audio array with shape {audio_array.shape} and sample rate {sample_rate}")

# Get audio and spectrogram (useful for visualization)
audio_array, sample_rate, spectrogram = tts.generate(
    text="This returns both audio and its spectrogram.",
    return_numpy=True,
    return_spectrogram=True,
    nfe_step=20,               # Number of denoising steps (32)
    cfg_strength=2.0,          # Classifier-free guidance strength (2)
    speed=1.0,                 # Speed of generated speech (1.0)
    cross_fade_duration=0.15,  # Cross-fade between chunks (0.15)
)
print(f"Generated spectrogram with shape {spectrogram.shape}")


In [ ]:
def save_spectrogram(spectrogram):
    plt.figure(figsize=(12, 4))
    plt.imshow(spectrogram, origin="lower", aspect="auto")
    plt.colorbar()
    plt.show()
    plt.close()


In [ ]:
save_spectrogram(spectrogram)